In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys
from datetime import datetime
from datetime import date

In [2]:
def getAndParseURL(url):
    #result = requests.get(url,headers = {"User-Agent": 'Mozilla/5.0'})
    #result = requests.get(url,headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'})
    result = requests.get(url,headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'})
    soup = bts(result.text, 'html.parser')
    return soup

In [3]:
product_counter = list()

html = getAndParseURL('https://www.amazon.co.uk/s?k=iphone+13&i=electronics&rh=n%3A5362060031%2Cp_n_feature_eight_browse-bin%3A14210458031%2Cp_72%3A419156031%2Cp_36%3A138898031%2Cp_89%3AApple%2Cp_n_feature_seven_browse-bin%3A5883356031&dc&page=2&qid=1674231701&rnid=5883351031&ref=sr_pg_2')

for counter in html.find_all("div",{"class":"a-section a-spacing-small a-spacing-top-small"}):
    product_counter.append(counter.text)
product_number = int(product_counter[0].split()[2])

product_page_number = 0
if product_number % 24 == 0:
    product_page_number = product_number // 24
else:
    product_page_number = product_number // 24 + 1

productCode_list = list()
productName_list = list()
productReviewNumber_list = list()
productStar_list = list()
for product_page in range(product_page_number):
#for product_page in range(3):
    html = getAndParseURL('https://www.amazon.co.uk/s?k=iphone+13&i=electronics&rh=n%3A5362060031%2Cp_n_feature_eight_browse-bin%3A14210458031%2Cp_72%3A419156031%2Cp_36%3A138898031%2Cp_89%3AApple%2Cp_n_feature_seven_browse-bin%3A5883356031&dc&page='+ str(product_page+1) +'&qid=1674231701&rnid=5883351031&ref=sr_pg_' + str(product_page+1))
    for product in html.find_all("div",{"data-component-type":"s-search-result"}):
        productCode_list.append(product["data-asin"])
    for name in html.find_all("span",{"class":"a-size-medium a-color-base a-text-normal"}):
        productName_list.append(name.text)
    temp = list()
    for review_number in html.find_all("span",{"class":"a-size-base s-underline-text"}):
        if product_page != 0:
            productReviewNumber_list.append(review_number.text)
        else:
            temp.append(review_number.text)
    if len(temp) > 0:
        for i in range(24):
            productReviewNumber_list.append(temp[i])
    temp = list()
    for product_star in html.find_all("div",{"class":"a-row a-size-small"}):
        temp.append(float(product_star.text[:3]))
    if product_page != (product_page_number - 1):
        for j in range(24):
                productStar_list.append(temp[j])
    else:
        for j in range(product_number % 24):
                productStar_list.append(temp[j])        
    print(product_page,len(productCode_list),len(productName_list),len(productReviewNumber_list),
          len(productStar_list))


0 24 24 24 24
1 48 48 48 48
2 63 63 63 63


In [4]:
import pandas as pd


df_products = pd.DataFrame(list(zip(productCode_list, productName_list, productReviewNumber_list,productStar_list)),
                  columns=["productCode", "productName", "productReviewNumber","productStar"])
df_products

,productCode,productName,productReviewNumber,productStar
0,B09G95MCDT,Apple iPhone 13 (128GB) - Midnight,"2,505",4.7
1,B09G9HQHRH,Apple iPhone 13 (128GB) - Blue,"2,505",4.7
2,B09V43FPYG,Apple iPhone 13 (128 GB) - Green,"2,505",4.7
3,B09G9FB7LV,Apple iPhone 13 (128GB) - Starlight,"2,505",4.7
4,B09MGFQ684,"Apple iPhone 13 Pro, 128GB, Sierra Blue (Renewed)",163,4.4
...,...,...,...,...
58,B0BNLZSY18,Apple iPhone 14 Pro (128 GB) - Deep Purple (Re...,1,1.0
59,B08L5SRBZ7,Apple iPhone 12 Pro (128GB) - Graphite,"1,237",4.7
60,B08L6Z1M4Y,Apple iPhone 11 (128GB) - Black,"2,397",4.7
61,B08PCFV74N,"Apple iPhone 12, 128GB, Green (Renewed)","1,403",4.3


In [5]:
df_products.to_csv("amazon_iphone13_products2.csv")

In [6]:
star_list = list()
review_list = list()
productcode_list = list()

In [ ]:
for productCode in range(len(productCode_list)):
    html = getAndParseURL('https://www.amazon.co.uk/product-reviews/'+str(productCode_list[productCode])+'/ref=cm_cr_getr_d_paging_btm_next_5?ie=UTF8&reviewerType=all_reviews&pageNumber=1')
    review_number = 0
    page_number = 0
    for review in html.find_all("div",{"data-hook":"cr-filter-info-review-rating-count"}):
        review_number = int(review.text.split()[3])
    if review_number % 10 == 0:
        page_number = review_number // 10
    else:
        page_number = review_number // 10 + 1

    for page in range(page_number):
        html = getAndParseURL('https://www.amazon.co.uk/product-reviews/'+str(productCode_list[productCode])+'/ref=cm_cr_getr_d_paging_btm_next_5?ie=UTF8&reviewerType=all_reviews&pageNumber=' + str(page+1))
        temp = list()
        if review_number % 10 == 0:
            temp_number = 10
        else:
            temp_number = review_number % 10
        for star in html.select('i[data-hook*="review-star-rating"]'):
            temp.append(int(star.get_text()[0]))
        if page != page_number-1:
            for i in range(len(temp)-10,len(temp)):
                star_list.append(temp[i])
        else:
            for i in range(len(temp)-temp_number,len(temp)):
                star_list.append(temp[i])        
        for review in html.find_all("span",{"data-hook":"review-body"}):
            review_list.append(review.text)
            productcode_list.append(productCode_list[productCode])
            
        print(productCode,page,len(star_list),len(review_list))
        if len(star_list) != len(review_list):
            break

0 0 10 10
0 1 20 20
0 2 30 30
0 3 40 40
0 4 50 50
0 5 60 60
0 6 70 70
0 7 80 80
0 8 90 90
0 9 100 100
0 10 110 110
0 11 120 120
0 12 130 130
0 13 140 140
0 14 150 150
0 15 160 160
0 16 170 170
0 17 180 180
0 18 190 190
0 19 200 200
0 20 210 210
0 21 214 214
1 0 224 224
1 1 234 234
1 2 244 244
1 3 254 254
1 4 264 264
1 5 274 274
1 6 284 284
1 7 294 294
1 8 304 304
1 9 314 314
1 10 324 324
1 11 334 334
1 12 344 344
1 13 354 354
1 14 364 364
1 15 374 374
1 16 384 384
1 17 394 394
1 18 404 404
1 19 414 414
1 20 424 424
1 21 428 428
2 0 438 438
2 1 448 448
2 2 458 458
2 3 468 468
2 4 478 478
2 5 488 488
2 6 498 498
2 7 508 508
2 8 518 518
2 9 528 528
2 10 538 538
2 11 548 548
2 12 558 558
2 13 568 568
2 14 578 578
2 15 588 588
2 16 598 598
2 17 608 608
2 18 618 618
2 19 628 628
2 20 638 638
2 21 642 642
3 0 652 652
3 1 662 662
3 2 672 672
3 3 682 682
3 4 692 692
3 5 702 702
3 6 712 712
3 7 722 722
3 8 732 732
3 9 742 742
3 10 752 752
3 11 762 762
3 12 772 772
3 13 782 782
3 14 792 792
3 15 

In [ ]:
import pandas as pd


df_reviews = pd.DataFrame(list(zip(productcode_list, star_list, review_list)),
                  columns=["productCode", "star", "review"])
df_reviews

In [ ]:
df_reviews.info()

In [ ]:
df_reviews.to_csv("amazon_final2.csv")